# Hierarchical Clustering Project

In this notebook, we will explore Hierarchical Clustering using a dataset from the [Datasets repository](https://github.com/MainakRepositor/Datasets). We will provide an overview of hierarchical clustering, its theoretical background, and demonstrate a complete data analysis workflow including preprocessing, model training, evaluation, and a comparative study using different linkage methods and distance metrics.

### Introduction
Hierarchical clustering is an unsupervised learning method that seeks to build a hierarchy of clusters without requiring the number of clusters to be specified beforehand (though it can be interpreted later from a dendrogram). We primarily distinguish between two approaches:
- **Agglomerative (Bottom-Up) Clustering:** Start with each data point as its own cluster and iteratively merge clusters.
- **Divisive (Top-Down) Clustering:** Start with one cluster containing all points and iteratively split it into smaller clusters.

**Significance & Applications:**
- **Marketing & Customer Segmentation:** Identify groups of similar customers to personalize marketing strategies.
- **Genealogy & Evolutionary Studies:** Analyze common ancestors and build phylogenetic trees.
- **Social Network Analysis:** Group similar users or communities.
- **Text/Document Clustering:** Group similar documents to organize information.

Hierarchical clustering is particularly helpful when we need to visualize and interpret the nested structure of data. The dendrograms offer insights into how clusters form at various distance thresholds.

### Dataset Selection & Description
For this project, we'll use **Mall_Customers.csv** from the [Datasets repository](https://github.com/MainakRepositor/Datasets). This dataset typically contains information on customers, including:
- **CustomerID**: Unique identifier
- **Genre**: Categorical feature
- **Age**: Numeric feature
- **Annual Income (k$)**: Numeric feature
- **Spending Score (1-100)**: Numeric feature

**Relevance for Hierarchical Clustering**
This dataset is popular for clustering tasks (especially in marketing segmentation). Customers can be grouped based on similar traits like annual income and spending habits, allowing businesses to target them more effectively. Hierarchical Clustering helps derive an intuitive dendrogram that can reveal different cluster structures at various similarity thresholds.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster

# Load the dataset
data_path = 'https://raw.githubusercontent.com/MainakRepositor/Datasets/refs/heads/master/Mall_Customers.csv'

df = pd.read_csv(data_path)
df.head()

### Exploratory Data Analysis (EDA)
We will:
- Print statistical summaries of the dataset.
- Visualize distributions and pairwise relationships.
- Detect patterns and outliers in the data.

In [ ]:
# Statistical summary
df.info()

In [ ]:
# Numeric columns summary
df.describe()

In [ ]:
# EDA visualizations
plt.figure(figsize=(10,4))
sns.countplot(x='Genre', data=df)
plt.title('Genre Distribution')
plt.show()

In [ ]:
# Histograms for numerical features
numeric_cols = ['Age','Annual Income (k$)','Spending Score (1-100)']
df[numeric_cols].hist(bins=10, figsize=(12,5))
plt.tight_layout()
plt.show()

In [ ]:
# Pairplot
sns.pairplot(df[numeric_cols])
plt.show()

In [ ]:
# Correlation heatmap
plt.figure(figsize=(6,5))
sns.heatmap(df[numeric_cols].corr(), annot=True, cmap='Blues')
plt.title('Correlation Heatmap')
plt.show()

### Data Preprocessing
1. **Handling Missing Values:** Drop or impute missing values if they exist.
2. **Encoding Categorical Variables:** Convert genre to numeric if needed.
3. **Scaling/Normalizing Features:** Apply standard scaling or normalization to ensure equal influence of features in distance calculations.

In [ ]:
# Check for missing values
missing_values = df.isnull().sum()
print('Missing Values in Each Column:\n', missing_values)

In [38]:
# For demonstration, assume minimal or no missing data; otherwise we'd address it.
df.dropna(inplace=True)  # simple drop if minimal missing data

# 2. Encoding categorical variables (if required)
df['Genre'] = df['Genre'].map({'Male': 0, 'Female': 1})

# 3. Selecting features relevant for clustering
X = df[['Genre','Age','Annual Income (k$)','Spending Score (1-100)']].copy()


#### Note on Genre Encoding

We encode the `Genre` column as `{'Male': 0, 'Female': 1}`. This encoding is arbitrary and does not imply any ordinal relationship between the categories. In unsupervised clustering, such encodings are often used to allow algorithms to process categorical data, but the numeric values themselves do not carry inherent meaning. The choice of encoding can influence clustering results, so it is important to interpret clusters with this in mind.

In [ ]:
# Scaling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled = pd.DataFrame(X_scaled, columns=X.columns)

X_scaled.head()

#### Including Categorical Variables in Distance-Based Clustering

In this analysis, we include the encoded `Genre` variable alongside numerical features when performing clustering. While this is a common approach, it is important to note that distance-based clustering algorithms (like hierarchical clustering) can be sensitive to the inclusion of categorical variables, especially when they are encoded as numbers. This can sometimes distort the distance calculations, as the algorithm may interpret the difference between categories as a meaningful numeric distance. In some cases, it may be preferable to exclude categorical variables or use specialized distance metrics that handle mixed data types. Here, we proceed with the inclusion for demonstration, but results should be interpreted with caution.

### Mathematical Explanation
**Agglomerative Clustering**
- Start with each point as its own cluster.
- At each iteration, merge the two "closest" clusters according to a linkage criterion.

**Linkage Criteria**
- **Single Linkage:** Distance between two clusters = minimum distance between any members.
- **Complete Linkage:** Distance between two clusters = maximum distance between any members.
- **Average Linkage:** Distance between two clusters = average pairwise distance between members.
- **Ward's Method:** Minimizes the variance within each cluster.

**Distance Metrics**
- **Euclidean Distance:** $d(\mathbf{x}, \mathbf{y}) = \sqrt{\sum (x_i - y_i)^2}$
- **Manhattan Distance:** $d(\mathbf{x}, \mathbf{y}) = \sum |x_i - y_i|$
- Other metrics exist (e.g., Chebyshev, Minkowski) but Euclidean is most common.

A dendrogram visualizes these merges. The vertical axis represents distance (or dissimilarity), and cutting the dendrogram at some level selects the desired number of clusters.

#### Why Hierarchical Clustering for This Dataset?

Hierarchical clustering is particularly well-suited for exploratory analysis when the number of clusters is not known in advance. The Mall Customers dataset is commonly used for customer segmentation, where the underlying group structure is not predefined. Hierarchical clustering provides a dendrogram, which visually represents how clusters are formed at different distance thresholds, allowing us to explore the data's natural grouping and select an appropriate number of clusters based on the data itself.

### Model Training & Evaluation
We'll implement agglomerative hierarchical clustering with various linkage methods and distance metrics.

Steps:
1. Generate the linkage matrix using `scipy.cluster.hierarchy.linkage`.
2. Plot the dendrogram to decide on the number of clusters.
3. Use `fcluster` (or manually cut the dendrogram) to assign cluster labels.
4. Evaluate the clusters using metrics like Silhouette Score.

In [ ]:
# Example with Ward's linkage and Euclidean distance
linked_ward = linkage(X_scaled, method='ward')

plt.figure(figsize=(10, 6))
dendrogram(linked_ward, truncate_mode='lastp', p=20)
plt.title('Dendrogram (Ward Linkage)')
plt.xlabel('Data Points')
plt.ylabel('Euclidean Distance')
plt.show()

In [ ]:
# Decide on a number of clusters by visually inspecting the dendrogram
# Here, we assume ~5 clusters visually.
labels_ward = fcluster(linked_ward, t=5, criterion='maxclust')

print('Assigned Cluster Labels (Ward):', np.unique(labels_ward))

# Evaluate with silhouette score
score_ward = silhouette_score(X_scaled, labels_ward)
print(f'Silhouette Score (Ward Linkage): {score_ward:.3f}')

#### Understanding the Silhouette Score

The silhouette score is a metric that quantifies how well each data point fits within its assigned cluster compared to other clusters. It ranges from -1 to 1:
- A score close to 1 indicates that the data point is well-matched to its own cluster and poorly matched to neighboring clusters.
- A score near 0 suggests that the data point is on or very close to the decision boundary between two clusters.
- A negative score indicates that the data point may have been assigned to the wrong cluster.

The average silhouette score across all data points provides an overall measure of clustering quality: higher values indicate better-defined clusters.

#### 2D Visualization of Clusters

To better visualize the clustering results, we use Principal Component Analysis (PCA) to project the data into two dimensions. Each point is colored according to its assigned cluster label. This helps us see the separation and cohesion of clusters in a reduced feature space.

In [ ]:
from sklearn.decomposition import PCA

# Reduce to 2D for visualization
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

plt.figure(figsize=(8,6))
scatter = plt.scatter(X_pca[:,0], X_pca[:,1], c=labels_ward, cmap='tab10', s=50, alpha=0.7)
plt.title('2D PCA Projection of Clusters (Ward Linkage)')
plt.xlabel('PCA Component 1')
plt.ylabel('PCA Component 2')
plt.colorbar(scatter, label='Cluster Label')
plt.show()

### Model Analysis & Visualization
In hierarchical clustering, **dendrograms** are crucial for understanding how clusters merge. We can also visualize the resulting clusters in various ways:
- **Heatmaps** to observe cluster structure.
- **Silhouette Coefficients** to check how well each data point fits into its assigned cluster.

Below, we showcase an example cluster heatmap (by reorganizing the data according to cluster labels).

In [ ]:
# Sort data by cluster labels for a cluster heatmap
df_analysis = X_scaled.copy()
df_analysis['Cluster'] = labels_ward
df_ordered = df_analysis.sort_values('Cluster')

plt.figure(figsize=(8, 6))
sns.heatmap(df_ordered.drop('Cluster', axis=1), cmap='viridis')
plt.title('Heatmap of Scaled Features Ordered by Ward Cluster Labels')
plt.show()

### Comparative Study
We will compare different linkage methods (Single, Complete, Average, Ward) and different distance metrics (e.g., Euclidean, Manhattan and cosine). Then we will compute the silhouette score for each approach.

A simple approach:
1. Iterate over linkages and metrics.
2. Generate dendrograms (optionally) or directly compute cluster labels using a chosen cutoff.
3. Compute silhouette scores for each combination.

In [ ]:
linkage_methods = ['single','complete','average','ward']
distance_metrics = ['euclidean','cityblock', 'cosine']  # cityblock is Manhattan
results = []

for method in linkage_methods:
    for metric in distance_metrics:
        # Ward linkage isn't compatible with metrics other than Euclidean.
        # So if method='ward', skip if metric!='euclidean'
        if method == 'ward' and metric != 'euclidean':
            continue

        Z = linkage(X_scaled, method=method, metric=metric)
        # We'll pick the same cluster count (5) for consistency.
        labels = fcluster(Z, t=5, criterion='maxclust')
        score = silhouette_score(X_scaled, labels, metric=metric)
        results.append((method, metric, score))

print('Method | Metric | Silhouette Score')
for r in results:
    print(f'{r[0]:<10} | {r[1]:<9} | {r[2]:.3f}')

#### Comparative Silhouette Scores

The table and barplot below summarize the silhouette scores for different combinations of linkage methods and distance metrics. This visual comparison helps identify which clustering approach yields the most cohesive and well-separated clusters for this dataset.

In [ ]:
# Present results as DataFrame and plot
results_df = pd.DataFrame(results, columns=['Linkage Method', 'Distance Metric', 'Silhouette Score'])

# Display the DataFrame
display(results_df)

# Barplot for visual comparison
plt.figure(figsize=(10,6))
sns.barplot(
    data=results_df,
    x='Linkage Method',
    y='Silhouette Score',
    hue='Distance Metric'
)
plt.title('Comparative Silhouette Scores by Linkage and Distance Metric')
plt.ylabel('Silhouette Score')
plt.xlabel('Linkage Method')
plt.legend(title='Distance Metric')
plt.tight_layout()
plt.show()

### Discussion & Conclusion
**Key Takeaways**
- Different linkage methods can produce significantly different cluster structures. Average's method often performs well when Cosine distance is used.
- The Silhouette Score provides a quantitative measure of how cohesive and separated the clusters are.
- For this particular dataset, you might find that certain linkage methods (e.g., Average) with Cosine distance yield higher silhouette scores.

**Potential Applications**
- **Market Segmentation:** Identify customer groups for targeted marketing.
- **Social Network Analysis:** Detect communities.
- **Genealogy and Phylogenetics:** Understand ancestral relationships.

**Limitations & Future Work**
- **Scalability:** Hierarchical clustering can be computationally expensive for very large datasets.
- **Distance Metric Sensitivity:** The choice of distance metric can drastically affect results.
- **Automated Cluster Selection:** Deciding the number of clusters from a dendrogram can be subjective.
- **Future**: Investigate advanced hierarchical methods, or combine hierarchical clustering with other clustering algorithms (e.g., K-means) for different stages of analysis.

### References
- [Scikit-Learn Documentation](https://scikit-learn.org/stable/modules/clustering.html#hierarchical-clustering)
- [Mall Customers Dataset](https://github.com/MainakRepositor/Datasets)
- M. R. Anderberg, _Cluster Analysis for Applications_, Academic Press, 1973.
- L. Kaufman and P. J. Rousseeuw, _Finding Groups in Data: An Introduction to Cluster Analysis_, Wiley, 1990.